# Assessing Fire and Rescue Service Strain to Anticipate Emerging Service Demands

## Files Required:

The following files (all located in the 'Data' folder) are required for the notebook to run:

*   2012_2021_NewData.xlsx
*   Acutal_ValueBY_Year.csv
*   HomeType.csv
*   FIREDISTRICT.shp **
*   FIRESTATIONS.shp **
*   FIRESTATIONZONES.shp **
*   Parcel.shp **
*   Blockgroups.shp **
*   Population files *
      *   Contains 9 csv files for 2013-2021
*   Homes65+ files *
      *   Contains 9 csv files for 2013-2021
*   Blockgroups files *
      *   Contains 10 .shp files


<br>
<br>
<br>

*Entire folder is required



** For all shapefiles `.shp` the corresponding `.cpg`, `.dbf`, `.prj`, `.shx`, `.xml` are required

# Ok time to play with code:

In [ ]:
!pip install geopandas -q
!pip install mapclassify -q
!pip install contextily -q
!pip install cpi -q

In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import mapclassify
import contextily as cx
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import seaborn as sns
from sklearn.linear_model import LinearRegression
from cpi import inflate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm
from calendar import month_abbr
from functools import reduce
import scipy as sp
from statsmodels.formula.api import ols

In [ ]:
drive.mount('/content/drive')

In [ ]:
cwd = os.getcwd()
pathCRFR = cwd + '/drive/My Drive/Colab Notebooks/'
pathFournier = 'Math Clinic/2023sp/CRFR/Data/'
if os.path.exists(pathCRFR + pathFournier):
    pathCRFR += pathFournier
os.listdir(pathCRFR) # lists files in the drive to check

In [ ]:
filepath = pathCRFR